In [1]:
#import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt

import numpy as np
import sys
sys.path.insert(0,'/local2/home/tong/caffe-master/python')
import caffe

import os
from matplotlib import rc
rc('mathtext', default='regular')


In [2]:
caffe.set_mode_gpu()
caffe.set_device(0)


In [ ]:
#for training
solver = caffe.SGDSolver('/local2/home/tong/fashionRecommendation/models/fashionNet_7/fashion_solver_7.prototxt')
solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
#solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_7/training_record/fashion_params_7_***.caffemodel')
#solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_7/training_record/fashion_params_7_***.caffemodel')


In [ ]:
#for saving caffemodel
net = caffe.Net('/local2/home/tong/fashionRecommendation/models/fashionNet_7/fashion_deploy_7.prototxt', caffe.TEST)


In [ ]:
#verify solver.net & solver.test_nets[0]
solver.net.forward()
solver.test_nets[0].forward()

print(solver.net.blobs['accuracy'].data, 
      solver.test_nets[0].blobs['accuracy'].data)

if solver.test_nets[0].blobs['accuracy'].data > 0.3:
    print 'feasible!'


In [ ]:
#training solver.net
recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_7/training_record/'

one_twen_epoch = 100000

test_interval = one_twen_epoch * 2

k = 0
start_iter = 0
end_iter = start_iter + one_twen_epoch*20

params = net.params.keys()

###################################
#                                 #
#                                 #
###################################

# record train (accuracy, loss)
tr_accu = open(recordDir + 'train_accu.txt','a')
tr_loss = open(recordDir + 'train_loss.txt','a')
tr_accu.write('0 '+str(solver.net.blobs['accuracy'].data)+'\r\n')
tr_loss.write('0 '+str(solver.net.blobs['rank_Loss'].data)+'\r\n')
tr_accu.close()
tr_loss.close()

for i in range (start_iter,end_iter):
    # update parameters
    solver.step(1)
    #save caffemodel every 1/20 epoch (~ one_twen_epoch iters)
    if (i % (one_twen_epoch - 1) == 0):
        #copy parameters solver.net -> net
        source_params = {pr: (solver.net.params[pr][0].data,solver.net.params[pr][1].data) for pr in params}
        target_params = {pr: (net.params[pr][0].data,net.params[pr][1].data) for pr in params}
        for pr in params:
            target_params[pr][0][...] = source_params[pr][0] #weights
            target_params[pr][1][...] = source_params[pr][1] #bias
        #save caffemodel
        net.save(recordDir+'fashion_params_7_'+str(i+1)+'.caffemodel')
    # record train (accuracy, loss)
    tr_accu = open(recordDir + 'train_accu.txt','a')
    tr_loss = open(recordDir + 'train_loss.txt','a')
    tr_accu.write(str(i+1)+' '+str(solver.net.blobs['accuracy'].data)+'\r\n')
    tr_loss.write(str(i+1)+' '+str(solver.net.blobs['rank_Loss'].data)+'\r\n')
    tr_accu.close()
    tr_loss.close()
    # record test (accuracy, loss) every 1/10 epoch (~ test_interval iters)
    if (i % (test_interval) == 0):
        te_accu = open(recordDir + 'test_accu.txt','a')
        te_loss = open(recordDir + 'test_loss.txt','a')
        te_accu.write(str(i)+' '+str(solver.test_nets[0].blobs['accuracy'].data)+'\r\n')
        te_loss.write(str(i)+' '+str(solver.test_nets[0].blobs['rank_Loss'].data)+'\r\n')
        te_accu.close()
        te_loss.close()
        
        #visualization for discovering overfitting
        train_accu_whole = []
        train_loss_whole = []
        test_accu_whole = []
        test_loss_whole = []

        train_accu_whole = open(recordDir+'train_accu.txt').readlines() 
        train_loss_whole = open(recordDir+'train_loss.txt').readlines()
        test_accu_whole = open(recordDir+'test_accu.txt').readlines()
        test_loss_whole = open(recordDir+'test_loss.txt').readlines()

        train_iter_idx = []
        train_loss = []
        train_accuracy = []

        test_iter_idx = []
        test_loss = []
        test_accuracy = []

        for i in range(0, len(train_accu_whole)):
            #train_iter_idx
            train_iter_idx.append(int(train_accu_whole[i].strip('\r\n').split(' ')[0]))
            #train_loss
            train_loss.append(float(train_loss_whole[i].strip('\r\n').split(' ')[1]))
            #train_accuracy
            train_accuracy.append(float(train_accu_whole[i].strip('\r\n').split(' ')[1]))

        for i in range(0, len(test_accu_whole)):
            #test_iter_idx
            test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
            #test_loss
            test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
            #test_accuracy
            test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))

        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(train_iter_idx, train_loss, '--rp', label = 'Training Loss')
        ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Validation Loss')
        ax_right = ax_left.twinx()
        ax_right.plot(train_iter_idx, train_accuracy, '-bp', label = 'Training Accuracy')
        ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Validation Accuracy')

        # ask matplotlib for the plotted objects and their labels
        lines_left, labels_left = ax_left.get_legend_handles_labels()
        lines_right, labels_right = ax_right.get_legend_handles_labels()
        ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations (n)")
        ax_left.set_ylabel("Loss")
        ax_right.set_ylabel("Accuracy")
        ax_right.set_title("Hightest Validation Accuracy: {:.3f}, at Training Iteration: {}".format(max(test_accuracy), test_iter_idx[test_accuracy.index(max(test_accuracy))]))
        plt.savefig(recordDir+'record.png', bbox_inches='tight')        
        # ending criteria
        if solver.test_nets[0].blobs['accuracy'].data > 0.93:
            k = k + 1
            if (k > 10):
                print '\n\nValidation accuracy: {} > 0.93 counted for 10 times\n\n'.format(solver.test_nets[0].blobs['accuracy'].data)
                break
        

In [32]:
#visualization for discovering overfitting
import os
import numpy as np
#import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
from matplotlib import rc
rc('mathtext', default='regular')

#os.system('scp '+recordDir + 'train_accu.txt '+'/home/tonghe2/fashionRecommendation/models/fashionNet_7/training_record')
#os.system('scp '+recordDir + 'train_loss.txt '+'/home/tonghe2/fashionRecommendation/models/fashionNet_7/training_record')
#os.system('scp '+recordDir + 'test_accu.txt '+'/home/tonghe2/fashionRecommendation/models/fashionNet_7/training_record')
#os.system('scp '+recordDir + 'test_loss.txt '+'/home/tonghe2/fashionRecommendation/models/fashionNet_7/training_record')

train_accu_whole = []
train_loss_whole = []
test_accu_whole = []
test_loss_whole = []

train_accu_whole = open(recordDir+'train_accu.txt').readlines() 
train_loss_whole = open(recordDir+'train_loss.txt').readlines()
test_accu_whole = open(recordDir+'test_accu.txt').readlines()
test_loss_whole = open(recordDir+'test_loss.txt').readlines()

train_iter_idx = []
train_loss = []
train_accuracy = []

test_iter_idx = []
test_loss = []
test_accuracy = []

for i in range(0, len(train_accu_whole)):
    #train_iter_idx
    train_iter_idx.append(int(train_accu_whole[i].strip('\r\n').split(' ')[0]))
    #train_loss
    train_loss.append(float(train_loss_whole[i].strip('\r\n').split(' ')[1]))
    #train_accuracy
    train_accuracy.append(float(train_accu_whole[i].strip('\r\n').split(' ')[1]))
    
for i in range(0, len(test_accu_whole)):
    #test_iter_idx
    test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
    #test_loss
    test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
    #test_accuracy
    test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))

# for i in range(0, 20):
#     train_iter_idx.append(i)
#     train_loss.append(-2*i+1)
#     train_accuracy.append(1.5*i*i)

# for i in range(0, 10):
#     test_iter_idx.append(2*i)
#     test_loss.append(-1.5*i+3)
#     test_accuracy.append(0.7*i*i)

fig = plt.figure()
ax_left = fig.add_subplot(111)
ax_left.plot(train_iter_idx, train_loss, '--rp', label = 'Training Loss')
ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Validation Loss')
ax_right = ax_left.twinx()
ax_right.plot(train_iter_idx, train_accuracy, '-bp', label = 'Training Accuracy')
ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Validation Accuracy')

# ask matplotlib for the plotted objects and their labels
lines_left, labels_left = ax_left.get_legend_handles_labels()
lines_right, labels_right = ax_right.get_legend_handles_labels()
ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
ax_left.grid()
ax_left.set_xlabel("Training Iterations (n)")
ax_left.set_ylabel("Loss")
ax_right.set_ylabel("Accuracy")
ax_right.set_title("Hightest Validation Accuracy: {:.3f}, at Training Iteration: {}".format(max(test_accuracy), test_iter_idx[test_accuracy.index(max(test_accuracy))]))
plt.savefig(recordDir+'record.png', bbox_inches='tight')


In [ ]:
scp tong@ares.cs.ucla.edu:/local2/home/tong/fashionRecommendation/models/fashionNet_7/training_record/record.png /home/tonghe2/fashionRecommendation/models/fashionNet_7/training_record


In [ ]:
#copy parameters solver.net -> net
params = net.params.keys()
source_params = {pr: (solver.net.params[pr][0].data,solver.net.params[pr][1].data) for pr in params}
target_params = {pr: (net.params[pr][0].data,net.params[pr][1].data) for pr in params}
for pr in params:
    target_params[pr][0][...] = source_params[pr][0] #weights
    target_params[pr][1][...] = source_params[pr][1] #bias


In [ ]:
#verify params copy
print 'solver.net: ', solver.net.params['metric_fc1'][0].data
print 'net:', solver.net.params['metric_fc1'][0].data


In [ ]:
#save caffemodel
net.save(recordDir+'fashion_params_7_'+str(i+1)+'.caffemodel')


In [ ]:
#verify caffemodel
net_Verify = caffe.Net('/local2/home/tong/fashionRecommendation/models/fashionNet_7/fashion_deploy_7.prototxt',
                recordDir+'fashion_params_7_'+str(i+1)+'.caffemodel',
                caffe.TEST)


In [ ]:
#verify input data (posi outfit)

#input VGG_mean.jpg
blob = caffe.proto.caffe_pb2.BlobProto()
data = open('./data/VGG_mean.binaryproto', 'rb' ).read()
blob.ParseFromString(data)
VGG_mean = np.array( caffe.io.blobproto_to_array(blob) )
VGG_mean.astype(np.uint8)

#get data from blobs
top = np.array(solver.net.blobs['top'].data[0])
bottom = np.array(solver.net.blobs['bottom'].data[0])
shoes = np.array(solver.net.blobs['shoes'].data[0])
#change data type to np.uint8
top.astype(np.uint8)
bottom.astype(np.uint8)
shoes.astype(np.uint8)
#add VGG_mean
top = top + VGG_mean[0,]
bottom = bottom + VGG_mean[0,]
shoes = shoes + VGG_mean[0,]
#BGR to RGB
top = np.array([top[2,],top[1,],top[0,]])
bottom = np.array([bottom[2,],bottom[1,],bottom[0,]])
shoes = np.array([shoes[2,],shoes[1,],shoes[0,]])
#(C,H,W) to (H,W,C)
top = np.rollaxis(top,0,3)
bottom = np.rollaxis(bottom,0,3)
shoes = np.rollaxis(shoes,0,3)
#temp buffer for using Image.fromarray(*)
temp_top = np.zeros(top.shape,np.uint8)
temp_bottom = np.zeros(bottom.shape,np.uint8)
temp_shoes = np.zeros(shoes.shape,np.uint8)
#copy data to buffer
for i in range(0,top.shape[0]):
    for j in range(0,top.shape[1]):
        for k in range(0,top.shape[2]):
            temp_top[i,j,k] = top[i,j,k]
            temp_bottom[i,j,k] = bottom[i,j,k]
            temp_shoes[i,j,k] = shoes[i,j,k]
#np.array 2 image
import Image
img_top = Image.fromarray(temp_top)
img_bottom = Image.fromarray(temp_bottom)
img_shoes = Image.fromarray(temp_shoes)
#save images
img_top.save('./top.jpg')
img_bottom.save('./bottom.jpg')
img_shoes.save('./shoes.jpg')

#verify input data (nega outfit)
#get data from blobs
top = np.array(solver.test_nets[0].blobs['top_n'].data[49])
bottom = np.array(solver.test_nets[0].blobs['bottom_n'].data[49])
shoes = np.array(solver.test_nets[0].blobs['shoes_n'].data[49])
#change data type to np.uint8
top.astype(np.uint8)
bottom.astype(np.uint8)
shoes.astype(np.uint8)
#add VGG_mean
top = top + VGG_mean
bottom = bottom + VGG_mean
shoes = shoes + VGG_mean
#BGR to RGB
top = np.array([top[2,],top[1,],top[0,]])
bottom = np.array([bottom[2,],bottom[1,],bottom[0,]])
shoes = np.array([shoes[2,],shoes[1,],shoes[0,]])
#(C,H,W) to (H,W,C)
top = np.rollaxis(top,0,3)
bottom = np.rollaxis(bottom,0,3)
shoes = np.rollaxis(shoes,0,3)
#temp buffer for using Image.fromarray(*)
temp_top = np.zeros(top.shape,np.uint8)
temp_bottom = np.zeros(bottom.shape,np.uint8)
temp_shoes = np.zeros(shoes.shape,np.uint8)
#copy data to buffer
for i in range(0,top.shape[0]):
    for j in range(0,top.shape[1]):
        for k in range(0,top.shape[2]):
            temp_top[i,j,k] = top[i,j,k]
            temp_bottom[i,j,k] = bottom[i,j,k]
            temp_shoes[i,j,k] = shoes[i,j,k]
#np.array 2 image
import Image
img_top = Image.fromarray(temp_top)
img_bottom = Image.fromarray(temp_bottom)
img_shoes = Image.fromarray(temp_shoes)
#save images
img_top.save('./top_n.jpg')
img_bottom.save('./bottom_n.jpg')
img_shoes.save('./shoes_n.jpg')